### Comment:
It was fun to do the Klotski challenge, but it's more efficient to just brute force Klotski. It took me less than half a second to find all 25955 possible positions (counting every similar block as equal). The shortest solution has 116 single steps and every configuration can be reached in 137 steps.

With recursion and backtracking, I got my solution in 2 seconds, but it required 3873 single steps (or 20928 steps when I forgot counting every similar block as equal).

In [50]:
import operator
import numpy as np

In [51]:
config = [[[0, 3], [0, 4]], 
          [[1, 3], [2, 3], [1, 4], [2, 4]],
          [[3, 3], [3, 4]],
          [[0, 1], [0, 2]],
          [[1, 2], [2, 2]],
          [[3, 1], [3, 2]],
          [[1, 1]],
          [[2, 1]],
          [[0, 0]],
          [[3, 0]]]

easy_config = [[[0, 3], [0, 4]],
               [[1, 1], [2, 1], [1, 2], [2, 2]],
               [[3, 3], [3, 4]],
               [[0, 1], [0, 2]],
               [[1, 4], [2, 4]],
               [[3, 1], [3, 2]],
               [[1, 3]],
               [[2, 3]],
               [[1, 0]],
               [[2, 0]]]

In [52]:
class Board:
    def __init__(self, board_state, previous_move):
        self.board_state = list(board_state)
        self.previous_move = previous_move
        self.children = []
        
    def check_solved(self):
        if self.board_state[1] == [[1, 0], [2, 0], [1, 1], [2, 1]]:
            return True
        return False
    
    def move(self, piece, direction):
        return [list(map(operator.add, point, direction)) for point in self.board_state[piece]]
    
    def move_possible(self, piece, direction):
        possible_piece = self.move(piece, direction)
        # außerhalb des Feldes
        if (True in [x_coordinate > 3 or x_coordinate < 0 for 
                     x_coordinate in [point[0] for point in possible_piece]] or 
            True in [y_coordinate > 4 or y_coordinate < 0 for 
                     y_coordinate in [point[1] for point in possible_piece]]):
            return False
        # Überlappung
        for point in possible_piece:
            if point in self.board_state[piece]:
                pass
            elif 1 in [block.count(point) for block in self.board_state]:
                return False
        return True
    
    def add_child(self, obj):
        self.children.append(obj)

In [56]:
def generate_children(board):
    for iter_piece in range(10):
        for direction in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            if board.move_possible(iter_piece, direction): 
                child_state = list(board.board_state)
                child_state[iter_piece] = board.move(iter_piece, direction)
                new_move = list(board.previous_move)
                new_move.append((iter_piece, direction))
                board.add_child(Board(child_state, new_move))

In [78]:
def get_path(board):
    path = []
    for i in board.previous_move:
        if i[1] == [0, 1]:
            direction = 'up'
        elif i[1] == [0, -1]:
            direction = 'down'
        elif i[1] == [1, 0]:
            direction = 'right'
        elif i[1] == [-1, 0]:
            direction = 'left'
        path.append('piece %d moves %s' % (i[0], direction))
    return path

In [58]:
def build_and_check_tree(root):
    root = [root]
    current_layer = []
    history = []
    depth = 0
    while not any([board.check_solved() for board in root]):
        depth += 1
        for child in root:
            generate_children(child)
        for node in root:
            for child in node.children:
                if not compress(child.board_state) in history:
                    current_layer.append(child)
                    history.append(compress(child.board_state))
        print('depth: %d, boards in layer: %d' % (depth, len(current_layer)))
        root = list(current_layer)
        current_layer = []
    return root

In [59]:
def compress(input_pieces):
    len1 = []
    len2 = []
    len4 = []
    for i in range(len(input_pieces)):
        if len(input_pieces[i]) == 1:
            len1.append((input_pieces[i][0][0], input_pieces[i][0][1]))
            len1.sort(key = operator.itemgetter(0, 1))
        if len(input_pieces[i]) == 2:
            len2.append((input_pieces[i][0][0], input_pieces[i][0][1]))
            len2.sort(key = operator.itemgetter(0, 1))
        if len(input_pieces[i]) == 4:
            len4.append((input_pieces[i][0][0], input_pieces[i][0][1]))
    return [len1, len2, len4]

In [60]:
def draw_board(piece):
    board = np.full((5, 4), ' ')
    for i in range(10):
        # X-Wert
        for x in range(len(piece[i])):
            board[piece[i][x][1]][piece[i][x][0]] = i
    print(np.flip(np.fliplr(board)), "\n")

In [76]:
solutions = []
for i in build_and_check_tree(Board(config, [])):
    if i.check_solved():
        draw_board(i.board_state)
        solutions.append(i)

depth: 1, boards in layer: 4
depth: 2, boards in layer: 11
depth: 3, boards in layer: 13
depth: 4, boards in layer: 18
depth: 5, boards in layer: 16
depth: 6, boards in layer: 19
depth: 7, boards in layer: 20
depth: 8, boards in layer: 22
depth: 9, boards in layer: 32
depth: 10, boards in layer: 38
depth: 11, boards in layer: 42
depth: 12, boards in layer: 36
depth: 13, boards in layer: 34
depth: 14, boards in layer: 42
depth: 15, boards in layer: 40
depth: 16, boards in layer: 34
depth: 17, boards in layer: 30
depth: 18, boards in layer: 29
depth: 19, boards in layer: 36
depth: 20, boards in layer: 47
depth: 21, boards in layer: 60
depth: 22, boards in layer: 54
depth: 23, boards in layer: 48
depth: 24, boards in layer: 48
depth: 25, boards in layer: 39
depth: 26, boards in layer: 35
depth: 27, boards in layer: 45
depth: 28, boards in layer: 52
depth: 29, boards in layer: 68
depth: 30, boards in layer: 74
depth: 31, boards in layer: 69
depth: 32, boards in layer: 51
depth: 33, boards 

In [79]:
num_lösungen = 0
for i in solutions:
    if i.check_solved():
        with open('Lösungen/Lösung%d.txt' % num_lösungen, 'w') as f:
            for item in get_path(i):
                f.write("%s\n" % item)
            num_lösungen += 1